# End of Week 1 Exercise — Bug Review Panel

**Purpose:** Beyond simply getting LLMs to provide technical explanations, this exercise tests whether models can truly inhabit the roles we assign them through system prompts. Can a model convincingly reason like a junior developer who misses subtle bugs, versus a senior engineer who catches everything? Or will the model's training override the persona — responding with the same objective, all-knowing analysis regardless of the role it was told to play? The goal is to explore how well LLMs follow persona-based instructions, whether their analysis naturally shifts in depth and nuance across experience levels, and whether prompts can subjectively short-circuit a model's behavior or if it will default to its trained objectivity.

Three buggy Python code samples are presented to two LLMs (GPT via OpenRouter, Gemma3 via Ollama).  
Each model assumes **three different roles** — Junior, Intermediate, and Senior SWE — and analyzes every sample with:

- **Level**: Easy / Medium / Hard
- **Description**: What the code is trying to do
- **Buggy**: Yes / No
- **Bug**: Type, nature, and description
- **Solution**: Corrected code or optimizations

In [10]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [11]:
MODEL_GPT = "openai/gpt-4o-mini"
MODEL_OLLAMA = "llama3.2"

OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ROLES = ["Junior SWE", "Intermediate SWE", "Senior SWE"]

SYSTEM_PROMPTS = {
    "Junior SWE": (
        "You are a Junior Software Engineer with about 1 year of professional experience. "
        "You have a basic understanding of Python and can spot obvious syntax errors, "
        "but you might miss subtle logical bugs or edge cases. "
        "Analyze the given code samples and respond with the structured format requested."
    ),
    "Intermediate SWE": (
        "You are an Intermediate Software Engineer with 3-5 years of professional experience. "
        "You have solid Python skills and can catch both syntax and logical errors. "
        "You understand common algorithms and data structures well and can suggest improvements. "
        "Analyze the given code samples and respond with the structured format requested."
    ),
    "Senior SWE": (
        "You are a Senior Software Engineer with 10+ years of professional experience. "
        "You have deep expertise in Python, algorithms, and software design. You catch all bugs "
        "— syntax, logical, and edge cases — and also consider performance, readability, "
        "naming conventions, and best practices. "
        "Analyze the given code samples and respond with the structured format requested."
    ),
}

In [12]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key found — add your OpenRouter key to .env as OPENAI_API_KEY")
else:
    print("API key loaded successfully")

gpt_client = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=api_key)
ollama_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

print(f"GPT client    -> {OPENROUTER_BASE_URL} using {MODEL_GPT}")
print(f"Ollama client -> {OLLAMA_BASE_URL} using {MODEL_OLLAMA}")

API key loaded successfully
GPT client    -> https://openrouter.ai/api/v1 using openai/gpt-4o-mini
Ollama client -> http://localhost:11434/v1 using llama3.2


In [13]:
sample_one = """
def find_min(arr)
  min = arr[0]
 for i in range(1, len(arr):
    if arr[i]  min:
       min = arr[i]
  return min
"""

sample_two = """
def bubbl_sort(arra):
  n = len(arra)
  for i in range(n):
    for j in range(n - i - 1)
      if arra[j > arra[j + 1]:
        arra[j], arra[j + 1]  arra[j + 1], arra[j]

  retur arra
"""

sample_three = """
def permutations(string):
 result=[]
 if len(string)==1:
  reurn [string]
  for i in range(len(string):
   char=string[i]
   remaining=string[:1]+string[i+1]
   for permutation in permutations(remaining)
    result.append(char+permutation)
 return result
"""

user_prompt = f"""Analyze the following three Python code samples. For EACH sample, provide your analysis in this exact format:

### Sample [number] — [function name]
- **Level**: [Easy / Medium / Hard](Rank based on how you view it's difficulty)
- **Description**: [Brief summary of what the code is trying to do]
- **Buggy**: [Yes / No]
- **Bug**: [Type, nature, and description of any bug(s) found]
- **Solution**:
```python
[corrected code]
```

---

Here are the three code samples:

**Sample 1 — find_min:**
```python{sample_one}```

**Sample 2 — bubble_sort:**
```python{sample_two}```

**Sample 3 — permutations:**
```python{sample_three}```
"""

In [14]:
def stream_analysis(client, model, role):
    """Stream a code review from the given client/model acting as a specific role."""
    display(Markdown(f"---\n## {role} — `{model}`\n"))
    stream = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPTS[role]},
            {"role": "user", "content": user_prompt},
        ],
        stream=True,
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)

In [15]:
display(Markdown("# GPT Code Review"))
for role in ROLES:
    stream_analysis(gpt_client, MODEL_GPT, role)

# GPT Code Review

---
## Junior SWE — `openai/gpt-4o-mini`


### Sample 1 — find_min
- **Level**: Easy
- **Description**: The function is intended to find the minimum value in a given list (array) of numbers.
- **Buggy**: Yes
- **Bug**: 
  - **Type**: Syntax Error
  - **Nature**: There are missing colons and incorrect use of operators.
  - **Description**: The function header is missing a colon, there is a missing closing parenthesis in the `range` function, and the comparison operator in the `if` statement is missing an operator (should be `<`).
- **Solution**:
```python
def find_min(arr):
    min_value = arr[0]
    for i in range(1, len(arr)):
        if arr[i] < min_value:
            min_value = arr[i]
    return min_value
```

---

### Sample 2 — bubble_sort
- **Level**: Medium
- **Description**: This function is attempting to implement the bubble sort algorithm to sort a list of numbers in ascending order.
- **Buggy**: Yes
- **Bug**: 
  - **Type**: Syntax Error
  - **Nature**: Missing colons and misplacement of comparison operators.
  - **Description**: The outer loop is missing a colon, and the comparison in the `if` statement needs to be corrected to properly compare `arra[j]` and `arra[j + 1]`. The swapping line has a syntax error as it is missing the assignment operator `=`.
- **Solution**:
```python
def bubble_sort(arra):
    n = len(arra)
    for i in range(n):
        for j in range(n - i - 1):
            if arra[j] > arra[j + 1]:
                arra[j], arra[j + 1] = arra[j + 1], arra[j]
    return arra
```

---

### Sample 3 — permutations
- **Level**: Hard
- **Description**: This function aims to generate all permutations of the characters in a given string.
- **Buggy**: Yes
- **Bug**: 
  - **Type**: Syntax Error
  - **Nature**: Incorrect use of return statement and missing parentheses.
  - **Description**: There is a misspelling of `return` as `reurn`, a missing closing parenthesis on the `if` condition, incorrect slicing when generating `remaining`, and the nested loop needs to use proper indentation and parentheses for function calls.
- **Solution**:
```python
def permutations(string):
    result = []
    if len(string) == 1:
        return [string]
    for i in range(len(string)):
        char = string[i]
        remaining = string[:i] + string[i+1:]  # Corrected to slice correctly
        for permutation in permutations(remaining):
            result.append(char + permutation)
    return result
```

---
## Intermediate SWE — `openai/gpt-4o-mini`


### Sample 1 — find_min
- **Level**: Medium
- **Description**: This function is intended to find the minimum value from a given list of numbers.
- **Buggy**: Yes
- **Bug**: 
  - **Type**: Syntax error
  - **Nature**: Missing colon after function definition and incorrect parentheses in range and if statement.
  - **Description**: The function definition should end with a colon, the range function is missing a closing parenthesis, and there is a syntax error in the if statement condition where the comparison operator is missing.
- **Solution**:
```python
def find_min(arr):
    min = arr[0]
    for i in range(1, len(arr)):
        if arr[i] < min:  # Changed to use '<' operator
            min = arr[i]
    return min
```

---

### Sample 2 — bubble_sort
- **Level**: Medium
- **Description**: This function aims to sort a given list of numbers using the bubble sort algorithm.
- **Buggy**: Yes
- **Bug**: 
  - **Type**: Syntax error
  - **Nature**: Missing colons at the end of the function and inner loop declarations, misplaced brackets in the if statement.
  - **Description**: The function name is misspelled, the inner loop lacks a colon at the end, and the comparison in the if statement is incorrect due to misplaced brackets.
- **Solution**:
```python
def bubble_sort(arra):  # Corrected spelling of the function name
    n = len(arra)
    for i in range(n):
        for j in range(n - i - 1):  # Added missing colon
            if arra[j] > arra[j + 1]:  # Corrected comparison statement
                arra[j], arra[j + 1] = arra[j + 1], arra[j]  # Fixed assignment syntax

    return arra  # Corrected spelling of return
```

---

### Sample 3 — permutations
- **Level**: Hard
- **Description**: This function is designed to generate all permutations of a given string.
- **Buggy**: Yes
- **Bug**: 
  - **Type**: Syntax error
  - **Nature**: Missing return statement, incorrect slicing of string, and missing closing parentheses in the loop.
  - **Description**: `return` is misspelled, the string slicing does not properly concatenate the remaining characters, and there are missing parentheses in the for loop.
- **Solution**:
```python
def permutations(string):
    result = []
    if len(string) == 1:
        return [string]  # Fixed return statement
    for i in range(len(string)):
        char = string[i]
        remaining = string[:i] + string[i + 1:]  # Fixed string slicing
        for permutation in permutations(remaining):  # Added missing parentheses
            result.append(char + permutation)
    return result
```

---
## Senior SWE — `openai/gpt-4o-mini`


### Sample 1 — find_min
- **Level**: Easy
- **Description**: This function aims to find the minimum value in a given list or array.
- **Buggy**: Yes
- **Bug**: 
  - **Syntax Error**: The function definition is missing a colon (`:`) at the end of the `def` line. 
  - **Logical Error**: The condition `if arr[i] min:` is missing an operator (it should be `if arr[i] < min:`).
  - **Syntax Error**: The `for` loop has a mismatched parenthesis in `len(arr)`.
- **Solution**:
```python
def find_min(arr):
    min = arr[0]
    for i in range(1, len(arr)):
        if arr[i] < min:
            min = arr[i]
    return min
```

---

### Sample 2 — bubble_sort
- **Level**: Medium
- **Description**: This function attempts to implement the bubble sort algorithm to sort a list or array in ascending order.
- **Buggy**: Yes
- **Bug**: 
  - **Syntax Error**: The function name `bubbl_sort` is incorrect; it should be `bubble_sort`.
  - **Syntax Error**: The loop `for j in range(n - i - 1)` is missing a colon (`:`) at the end.
  - **Logical Error**: The condition `if arra[j > arra[j + 1]:` is incorrectly formed and should be `if arra[j] > arra[j + 1]:`.
  - **Syntax Error**: The swap syntax is incorrect (`arra[j], arra[j + 1]  arra[j + 1], arra[j]`) and missing an assignment operator. The correct statement should be `arra[j], arra[j + 1] = arra[j + 1], arra[j]`.
  - **Syntax Error**: The return statement has a typo; it should be `return arra`.
- **Solution**:
```python
def bubble_sort(arra):
    n = len(arra)
    for i in range(n):
        for j in range(n - i - 1):
            if arra[j] > arra[j + 1]:
                arra[j], arra[j + 1] = arra[j + 1], arra[j]
    return arra
```

---

### Sample 3 — permutations
- **Level**: Hard
- **Description**: This function generates all possible permutations of a given string.
- **Buggy**: Yes
- **Bug**: 
  - **Syntax Error**: The return statement is incorrectly spelled as `reurn` instead of `return`.
  - **Syntax Error**: The `for` loop is missing a colon (`:`) at the end.
  - **Logical Error**: The slice for `remaining` is incorrect; it uses `string[:1]` which should be `string[:i]` to slice up to the character before `i`.
  - **Syntax Error**: The line `for permutation in permutations(remaining)` is missing a colon (`:`) at the end.
- **Solution**:
```python
def permutations(string):
    result = []
    if len(string) == 1:
        return [string]
    for i in range(len(string)):
        char = string[i]
        remaining = string[:i] + string[i+1:]  # Correct slicing
        for permutation in permutations(remaining):  # Add colon
            result.append(char + permutation)
    return result
```

In [9]:
display(Markdown("# Ollama Code Review"))
for role in ROLES:
    stream_analysis(ollama_client, MODEL_OLLAMA, role)

# Ollama Code Review

---
## Junior SWE — `llama3.2`


### Sample 1 — find_min
- **Level**: Easy
- **Description**: This function finds the minimum value in a given array.
- **Buggy**: Yes
- **Bug**: 
  - Missing colon after `for` statement.
  - Use of single equals sign (`=`) for assignment, which is also unnecessary here. Python's strict syntax would allow this without issues, but it's generally good practice to use both sides of the assignment arrow (`=>`) or simply remove single equals signs for clarity.
- **Solution**:
```python
def find_min(arr):
  min = arr[0]
  for i in range(1, len(arr)):
    if arr[i] < min:
      min = arr[i]
  return min
```

### Sample 2 — bubble_sort
- **Level**: Hard
- **Description**: This function sorts an array using the Bubble Sort algorithm.
- **Buggy**: Yes
- **Bug**: 
  - Use of `j >` instead of `j <` for sorting loop. It should be less than (`j <`) to sort in ascending order, otherwise it will produce incorrect results and bubble out-of-order elements or go into an infinite loop under certain conditions.
  - Missing `)` at the end of the outer loop's range argument `range(n)`.
  - Incorrect comparison operator due to lack of space. It should be `arr[j] > arra[j+1]` not just compare `arra[j]` and `arra[j + 1]`, also the swap line was swapped (in the swap function there is no j+1).
- **Solution**:
```python
def bubble_sort(arr):
  n = len(arr)
  for i in range(n):
    for j in range(n - i - 1):  
      if arr[j] > arr[j + 1]:
        arr[j], arr[j + 1] = arr[j + 1], arr[j]
  
  return arr
```

### Sample 3 — permutations
- **Level**: Easy
- **Description**: This function generates all possible permutations of a given string.
- **Buggy**: Yes
- **Bug**:
  - The initial code for `remaining` is incorrect. It should be `${string[:i]} + string[i+1:]` to exclude the current character, not `${string[:1]} + string[i+1:]`, which will exclude everything except the first character.
- **Solution**:
```python
def permutations(string):
  result = []
  if len(string) == 1:
    return [string]
  for i in range(len(string)):
    char = string[i]
    remaining = string[:i] + string[i+1:]
    for permutation in permutations(remaining):
      result.append(char + permutation)
  return result
```

---
## Intermediate SWE — `llama3.2`


### Sample 1 — find_min:

- **Level**: Medium
- **Description**: This function is supposed to find the minimum element in a sorted array.
- **Buggy**: Yes
- **Bug**: 
  - `arr` should be passed as an argument, not defined inside the function. It's also mentioned as "arr" but used as "arr" consistently throughout the code. The variable name seems arbitrary and could be improved to something like `numbers`.
  - The condition in the if statement is inconsistent using single equals sign for comparison in some places (`arr[i] < min`) and double equals sign in others (e.g., `min = arr`). This should be consistent.
- **Solution**:
```python
def find_min(numbers):
  min = numbers[0]
  for i in range(1, len(numbers)):
    if numbers[i] < min:
      min = numbers[i]
  return min
```

---

### Sample 2 — bubble_sort:

- **Level**: Hard
- **Description**: This function seems to be implementing the Bubble Sort algorithm. 
- **Buggy**: Yes
- **Bug**:
  - The range function call in the inner loop has a syntax error, missing a colon at the end of the line. The correct syntax should be `range(n - i - 1)` not `range(n - i - 1)`. The parentheses and double quote at the end are extra.
  - In the swap part inside the if statement `arra[j] arrra[j+1]`, 's seem to be swapped but in standard bubble sort algorithm, j would be updated. This logic is incorrect according Bubble Sort algorithm
- **Solution**:
```python
def bubble_sort(arr):
  n = len(arr)
  for i in range(n):
    for j in range(n - i - 1):
      if arr[j] > arr[j + 1]:
        arr[j], arr[j + 1] = arr[j + 1], arr[j]
  return arr
```

---

### Sample 3 — permutations:

- **Level**: Hard
- **Description**: This function seems to be generating all permutations of a given string.
- **Buggy**: Yes
- **Bug**:
  - The slicing operation in the `remaining` variable is incorrect. It should be `string[i+1:]` not `string[:i+1]`. This would exclude the character at index `i` instead of including it.
  - There's also a syntax error in the line inside the for permutation loop. 'reurn' should be 'return'.
- **Solution**:
```python
def permutations(string):
  result = []
  if len(string) == 1:
    return [string]
  for i in range(len(string)):
    char = string[i]
    remaining = string[:i] + string[i+1:]
    for permutation in permutations(remaining):
      result.append(char + permutation)
  return result
```

---
## Senior SWE — `llama3.2`


### Sample 1 — find_min

- **Level**: Easy
- **Description**: This function is intended to find the minimum element in a given list of numbers.
- **Buggy**: Yes
- **Bug**: 
	* Incorrect syntax: `def` keyword is missing after `find_min`, should be `def`.
	* Indentation error: The closing bracket for the outer loop and inner loop was missing, also the correct syntax for range function should include a colon at the end.
- **Solution**:
```python
def find_min(arr):
  min = arr[0]
  for i in range(1, len(arr)):
    if arr[i] < min:
      min = arr[i]
  return min
```

### Sample 2 — bubble_sort

- **Level**: Hard
- **Description**: This function is intended to perform the Bubble Sort algorithm on a given array of numbers.
- **Buggy**: Yes
- **Bug**:
	* Incorrect variable name: `arra` should be replaced with `arra` as per Python convention for variable names.
	* Syntax error in comparison operator: `ARRA[j > arra[j + 1]` is not valid syntax. Instead, it should use `ARRA[j] < arra[j + 1]`.
	* Syntax error in assignment statement: Incorrect order of variables being swapped.
- **Solution**:
```python
def bubble_sort(arra):
  n = len(arra)
  for i in range(n):
    for j in range(n - i - 1):
      if arra[j] > arra[j + 1]:
        arra[j], arra[j + 1] = arra[j + 1], arra[j]
  return arra
```

### Sample 3 — permutations

- **Level**: Medium
- **Description**: This function is intended to generate all permutations of a given string.
- **Buggy**: Yes
- **Bug**:
	* Incorrect syntax: The `return` keyword should be followed by an expression, not just the words "return".
	* Syntax error in slicing string: The line `remaining=string[:1]+string[i+1]` may raise an exception if `i` is equal to the last index of the string.
- **Solution**:
```python
def permutations(string):
  result = []
  if len(string) == 1:
    return [string]
  for i in range(len(string)):
    char = string[i]
    remaining = string[:i] + string[i+1:]
    for permutation in permutations(remaining):
      result.append(char + permutation)
  return result
```